In [22]:
import speech_recognition as sr
import numpy as np
import time
import pyttsx3

# parameters for listening
dB_threshold = 10000
t_wait = 1    # in seconds


def listen_command(threshold=dB_threshold, t_timeout=t_wait):
    """ () -> str or False
    Return the command the program listens. Return False if failed to listen/understand.
    """
    r = sr.Recognizer()
    r.dynamic_energy_threshold = threshold    # high threshold so that program only responds to large voice
    print("Listening...")
    # starts listening once threshold reached -> stops listening when quiet
    with sr.Microphone() as source:
        audio = r.listen(source, timeout=t_timeout)
    try:
        return r.recognize_google(audio).lower()
    except sr.UnknownValueError or sr.RequestError:
        return False

In [13]:
def process_command(command):
    words = np.array(command.split(' '))

    ident = ''
    action = 0
    
    if 'read' in words and 'recipe' in words:
        #action 1 means read the recipe
        action = 1
        ind = np.where(words == 'recipe')[0]
        if ind < len(words):
            ident = words[ind+1][0]
    elif 'next' in words:
        #action 2 means read the instruction
        action = 2
#         ind = np.where(words == 'instruction')[0]
#         if ind < len(words):
#             ident = words[ind+1][0]
    elif 'stop' in words:
        #action 3 means stop
        action = 3
    elif 'repeat' in words:
        #action 4 means repeat recipe or instruction
        action = 4
    elif 'ingredient' in words:
        #fetch the ingredient
        ind = np.where(words == 'ingredient')[0]
        if ind < len(words):
            ident = words[ind+1][0]
        action = 5
    else:
        #action 6 means dont understand - help the user with instructions
        action = 6
    
    
    return action, ident

In [8]:
######## TEST BLOCK ########
command = str(listen_command())
print('Command: '+ command) 
process_command(command)

Listening...
Command: read recipe 1


(1, '1')

In [18]:
idle = 1000
def listen(wake_word, t_wait=idle):
    waiting = True
    start = time.time()
    
    while waiting and time.time()-start < t_wait:
        words = str(listen_command()).split(' ')
        if wake_word in words:
            waiting = False
    
    return True

In [15]:
######## TEST BLOCK ########
print(listen('hello'))

Listening...
Listening...
True


In [16]:
def activate(wake_word, instruction_book, ingredient_book):
    active = listen(wake_word)
    engine = pyttsx3.init()
    
    #machine need to tell user what to do
    welcome = 'what is up'
    engine.say(welcome)
    engine.runAndWait()
    
    #in
    recipe_id = ''
    ingredient_itr = 0
    instruction_itr = 0

    while active:
        command = str(listen_command())
        print('Command: '+ command)
        action, ident = process_command(command)
        
        #action 1: read the recipe
        if action == 1:
            if ident == '': # no recipe indicated in the instruction
                engine.say('expected recipe name right after recipe word')
                engine.runAndWait()
                break;
            # now, recipe is provided: identify the relevant instructions & ingredients
            recipe_id = ident
            instructions = instruction_book[ident] # this is a list of instructions
            ingredients = ingredient_book[ident]   # this is a dictionary of ingredients

            # make a list of ingredient keys
            L_ingredient_key = [*ingredients]
            
            #speak the ingredients
            key = L_ingredient_key[ingrediet_itr]
            engine.say(key+','+ingredients[key])
            ingredient_itr += 1
            engine.runAndWait()

            #speak the first instruction
            
            
        elif action == 2:
            if ingredient_itr < len(ingredients.keys()):
                key = L_ingredient_key[ingredient_itr]
                ingredient_itr += 1

                engine.say(key+','+ingredient[key])
                engine.runAndWait()
            elif instruction_itr < len(instructions):
                instruction_itr += 1

                engine.say(instructions[instruction_itr])
                engine.runAndWait()
            else:
                farewell = 'you done brother'
                engine.say(farewell)
                engine.runAndWait()
                active = False

            
            
        elif action == 3:
            engine.say('Stopped')
            engine.runAndWait()
            if listen(wake_word):
                engine.say('I am back')
                engine.runAndWait()
            

        elif action == 4: 
            if ingredient_itr < len(ingredients.keys())+1:
                key = L_ingredient_key[ingredient_itr-1]
                engine.say(key+','+ingredient[key])
                engine.runAndWait()

            else:
                engine.say(instructions[instruction_itr-1])
                engine.runAndWait()
            

        elif action == 5: 
            if ident == '':
                engine.say('expected recipe name right after recipe word')
                engine.runAndWait()
                break;
            
            key = ident
            engine.say(key+','+ingredients[key])
            engine.runAndWait()
            

        elif action == 6:
            
            confused = 'I do not understand'
            engine.say(confused)
            engine.runAndWait()
            
            

        

In [21]:
######## TEST BLOCK ########
instruction_book = {'apple': ['first do this', 'pour the water', 'lift the handle', 'stir']}
ingredient_book = {'apple': {'eggs': '1', 'beans': '2 pounds', 'jello': '1 packet'}}
activate('hello', instruction_book, ingredient_book)

Listening...


KeyboardInterrupt: 

In [37]:
instruction_book = {'apple': ['first do this', 'pour the water', 'lift the handle', 'stir']}
ingredient_book = {'apple': {'eggs': '1', 'beans': '2 pounds', 'jello': '1 packet'}}
a = ingredient_book['apple']

In [43]:
print(a.keys())

dict_keys(['eggs', 'beans', 'jello'])


In [10]:
d = {'a':1,'b':2,'c':3}
print([*d])

['a', 'b', 'c']
